[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milteam/unsupervised-summary-based-segmentation/blob/feature/update/notebooks/example_colorize.ipynb)

In [ ]:
# # for google colab
# !pip install -q datasets pathlib2 colorize hydra-core sentence_transformers optuna tqdm
# import nltk
# nltk.download('punkt')

# !python -m spacy download ru_core_news_sm

In [ ]:
!git clone --branch=feature/update https://github.com/milteam/unsupervised-summary-based-segmentation.git

In [4]:
import sys
sys.path.append('./')
sys.path.append('./lib/pipelines/')
sys.path.append('./lib/pipelines/utilities/')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DEVICES = 'cuda'

import torch
from datasets import Dataset
from lib.pipelines.utilities.dataset import load_dataset_by, colorize
from lib.pipelines.utilities.general import load_config_by, log, embed, calc_metric, get_sentence_encoder
from lib.pipelines.utilities.tiling import TopicTilingModel
from lib.pipelines.summary_integration import *

In [5]:
config_path = '../configs/'
arg_dict = {}
cfg = load_config_by(arg_dict, config_path)

In [6]:
text_1 = """
Клиент: алло да
Сотрудник: * добрый день это ваш менеджер *.  вам удобно сейчас говорить
Клиент: да я вас слушаю
Сотрудник: хотела уточнить мне недавно подали бизнес-карту скажите уже пробовали ее оплачивать
Клиент: нет
Сотрудник: на будущее вам напомню чтобы все возможности карты вам использовать на нее нужно будет лежит лимиты установить.  можете сделать самостоятельно в личном кабинете.  если лимиты на карту не установленный то она будет работать только на внесение наличных.  а я могу вам направить инструкцию с номера * как устанавливаю лимиты нужно будет
Клиент: ну я пока я не собираюсь еще пользовалась
Сотрудник: ну.  хорошо.  направлю и так же еще хотела уточнить подскажите планируете денежные средства переводить с расчетного счета наличную карту заинтересованы без комиссионном переводе
Клиент: ну конечно
Сотрудник: я хотела вам рассказать про сервис э зарплатный проект.  он позволяет предпринимателем переводить до * рублей дополнительно на свою личную карту.  по * коду код специально разработан для предпринимателей называется перевод средств и * наличные счет.  то есть здесь необязательно иметь сотрудника в штате зарплатный проект можно использовать для себя.  сервис бесплатный подключается также дистанционно.  также могу вам направить инструкцию по подключению
Клиент: хорошо.  хорошо
Сотрудник: угу
Клиент: да
Сотрудник: и.  также свои контакты вам направляю с номера * если вопросы какие-то будут возникает по обслуживанию обязательно обращайтесь буду рада помочь вам.  и по зарплатному проекту еще раз говорю что обязательно переводите по * * коду только тогда будет у вас перевод без комиссии на свою карту
Клиент: все я поняла
Сотрудник: благодарю вас сейчас за уделенное время всего доброго до свидания
Клиент: до свидания
"""

In [7]:
def get_dataset(texts, remove_utterances=True):
    def gen():
        for text in texts:
            if remove_utterances:
                text = text.replace('Сотрудник: ', '').replace('Клиент: ', '')
            text = text.strip().split('\n')
            yield {'sections': text}
    ds = Dataset.from_generator(gen)
    return ds

ds = get_dataset([text_1])

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
sentence_model = get_sentence_encoder(cfg)
ds = ds.map(embed, batched=True, batch_size=cfg.sbert.cpu_batch_size,
                        fn_kwargs={'model': sentence_model, 'gpu_batch_size': cfg.sbert.gpu_batch_size})

model_name = 'IlyaGusev/rut5_base_sum_gazeta'
# model_name = 'IlyaGusev/mbart_ru_sum_gazeta'

device = torch.device('cuda:0' if len(cfg.gpus) else 'cpu')
ds = ds.map(summarize, fn_kwargs={'model_name': model_name, 'device': device})

ds = ds.map(split_sent_batched, num_proc=16, fn_kwargs={'language': 'russian'})
ds = ds.map(embed_summary, batched=True, batch_size=cfg.sbert.cpu_batch_size,
                                    fn_kwargs={'model': sentence_model, 'gpu_batch_size': cfg.sbert.gpu_batch_size})
ds = ds.map(calculate_prob_vector, batch_size=cfg.batch_size,
                                        num_proc=cfg.num_cpu, batched=True)
ds = ds.with_format('numpy', columns=['embeddings', 'embeddings_summary', 'probs', 'summary', 'splitted_summary', 'sections'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [10]:
ds

Dataset({
    features: ['sections', 'embeddings', 'summary', 'splitted_summary', 'embeddings_summary', 'probs'],
    num_rows: 1
})

In [11]:
ds['sections']

array([['алло да',
        '* добрый день это ваш менеджер *.  вам удобно сейчас говорить',
        'да я вас слушаю',
        'хотела уточнить мне недавно подали бизнес-карту скажите уже пробовали ее оплачивать',
        'нет',
        'на будущее вам напомню чтобы все возможности карты вам использовать на нее нужно будет лежит лимиты установить.  можете сделать самостоятельно в личном кабинете.  если лимиты на карту не установленный то она будет работать только на внесение наличных.  а я могу вам направить инструкцию с номера * как устанавливаю лимиты нужно будет',
        'ну я пока я не собираюсь еще пользовалась',
        'ну.  хорошо.  направлю и так же еще хотела уточнить подскажите планируете денежные средства переводить с расчетного счета наличную карту заинтересованы без комиссионном переводе',
        'ну конечно',
        'я хотела вам рассказать про сервис э зарплатный проект.  он позволяет предпринимателем переводить до * рублей дополнительно на свою личную карту.  по * к

In [12]:
ds['summary']

array(['Мне недавно подали бизнес-карту и уже пробовали ее оплачивать. я также хотела рассказать о сервисе э зарплатный проект, который позволяет предпринимателям переводить до * рублей дополнительно на свою личную карту.'],
      dtype='<U213')

In [13]:
topictiling_parameters = {
    'window_size': 1, # probably most affecting hyperparameter
    'threshold': 0.64,
    'smoothing_passes': 1,
    'smoothing_window': 1,
    'n_smooth_savgol': 3,
    'savgol_k': 0.05, # approximately 1/expected_segment_length
    'polyorder': 3
}
tiling_model = TopicTilingModel(**topictiling_parameters)


def add_features(example):
    boundaries_pred = get_borders_sumseg(example, tiling_model, cfg.sumseg.tiling.plot)

    example['labels'] = [0] * len(example['sections'])  # for a stab
    example['boundaries_pred'] = boundaries_pred
    example['topics_pred'] = get_topics(boundaries_pred)
    return example

ds = ds.map(add_features)

Parameter 'function'=<function add_features at 0x7b6a0e19dfc0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [14]:
print(ds['boundaries_pred'])

['00000000001000000']


In [15]:
colorize(ds)

0
N of sections in document: 1
N of simple sentences in summary: 4
Summary: Мне недавно подали бизнес-карту и уже пробовали ее оплачивать. я также хотела рассказать о сервисе э зарплатный проект, который позволяет предпринимателям переводить до * рублей дополнительно на свою личную карту.
Summary split: ['Мне недавно подали бизнес-карту и уже пробовали ее оплачивать'
 'и уже пробовали ее'
 'я также хотела рассказать о сервисе э зарплатный проект, который позволяет предпринимателям переводить до * рублей дополнительно на свою личную карту'
 'сервисе э зарплатный']

Predicted:
Classes: 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
алло да
* добрый день это ваш менеджер *.  вам удобно сейчас говорить
да я вас слушаю
хотела уточнить мне недавно подали бизнес-карту скажите уже пробовали ее оплачивать
нет
на будущее вам напомню чтобы все возможности карты вам использовать на нее нужно будет лежит лимиты установить.  можете сделать самостоятельно в личном кабинете.  если лимиты на карту не установленный 